# Jupyter Notebook for mainnet deployment


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [ ]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3},
"sejong":  {"iconservice": "https://sejong.net.solidwallet.io", "nid": 83}
}

env = connections[network]


### Necessary packages 

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder, DepositTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result


### Required service

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [ ]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())


In [ ]:
# Run this in case you want to import previous contract addresses
contracts_file_custom = "reward_retest_contracts_20210702015446.pkl"
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

## Helper Contracts and Variables

In [ ]:
# Dont run this in case you have imported contracts from a pkl file 
usd = "usds" # "usds" or "usdb"
contracts = {}
PREP_LIST = ["hxec79e9c1c882632688f8c8f9a07832bcabe8be8f","hxd3be921dfe193cd49ed7494a53743044e3376cd3",    "hx9e7509f86ea3ba5c139161d6e92a3982659e9f30","hxaad52424d4aec9dac7d9f6796da527f471269d2c"]
TIMESTAMP = 1624634100000000 # 9pm NST 25th June

EMISSION_PER_DAY= 400000
SECONDS_PER_DAY = 86400

OICX_EMISSION = int(EMISSION_PER_DAY * 0.2 * 0.9 * 10 ** 18 // SECONDS_PER_DAY)
DICX_EMISSION = int(EMISSION_PER_DAY * 0.2 * 0.1 * 10 ** 18 // SECONDS_PER_DAY)
OUSDS_EMISSION = int(EMISSION_PER_DAY * 0.4 * 0.5 * 10 ** 18 // SECONDS_PER_DAY)
DUSDS_EMISSION = int(EMISSION_PER_DAY * 0.4 * 0.5 * 10 ** 18 // SECONDS_PER_DAY)
OIUSDC_EMISSION = int(EMISSION_PER_DAY * 0.4 * 0.5 * 10 ** 18 // SECONDS_PER_DAY)
DIUSDC_EMISSION = int(EMISSION_PER_DAY * 0.4 * 0.5 * 10 ** 18 // SECONDS_PER_DAY)


contracts['sicx']="cxae6334850f13dfd8b50f8544d5acb126bb8ef82d"
contracts['staking']="cx9d829396d887f9292d8af488fab78ad24ab6b99a"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
if usd == "usds":
    contracts[usd]="cxaa068556df80f9917ef146e889f0b2c4b13ab634"
elif usd == "usdb":
    contracts[usd] = "cxe875b38397fb08b7166acaa0bda74c85249f2eff"
contracts['bandOracle'] = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
contracts["dex"] = "cx399dea56cf199b1c9e43bead0f6a284bdecfbf62"

# Deploying all contracts


In [ ]:

deploy = ['addressProvider','daoFund','delegation','feeProvider','governance', 
          'lendingPool', 'lendingPoolCore','lendingPoolDataProvider',
          'liquidationManager','ommToken', 'priceOracle','rewardDistribution' ,
          'worker_token']

directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
        
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

In [ ]:
pprint(contracts)

## Deploying remaining contracts


In [ ]:
for item in deploy:
    if item not in contracts.keys():
        params = {}
        if item == "ommToken":
            params = {'_initialSupply':0, '_decimals': 18}
        elif item == "worker_token":
            params = {'_initialSupply':100, '_decimals': 18}
       
        deploy_transaction = DeployTransactionBuilder()\
            .from_(deployer_wallet.get_address())\
            .to(GOVERNANCE_ADDRESS)\
            .nid(NID)\
            .nonce(100)\
            .content_type("application/zip")\
            .content(gen_deploy_data_content(item))\
            .params(params)\
            .build()

        step_limit = icon_service.estimate_step(deploy_transaction) + 100000
        
        signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
        tx_hash = icon_service.send_transaction(signed_transaction)

        res = get_tx_result(tx_hash)
        print(res)
        contracts[item] = res.get('scoreAddress', '')

## Deploying otoken for USDS

In [ ]:

params = {"_name":"Omm USDS Interest Token","_symbol":"oUSDS"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDS'] = res.get('scoreAddress', '')



# contracts

## Deploying debt token for usds (dUSDS)

In [ ]:
params = {"_name":"Omm USDS Debt Token","_symbol":"dUSDS"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('dToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['dUSDS'] = res.get('scoreAddress', '')



# contracts

## Deploying otoken for icx

In [ ]:
#deploy oICX 
params = {"_name":"Omm ICX interest Token","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

# contracts

## Deploying debt token for icx (dICX)

In [ ]:

params = {"_name":"Omm ICX Debt Token","_symbol":"dICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('dToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['dICX'] = res.get('scoreAddress', '')



# contracts

## Deploying otoken for iusdc


In [ ]:
#deploy oIUSDC
params = {"_name":"Omm IUSDC Interest Token","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

# contracts

## Deploying debt token for iusdc dIUSDC

In [ ]:

params = {"_name":"Omm IUSDC Debt Token","_symbol":"dIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('dToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['dIUSDC'] = res.get('scoreAddress', '')



# contracts

## Deposit fee to Lending Pool

In [ ]:
call_transaction = DepositTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts["lendingPool"]) \
    .nid(3) \
    .value(5000 * 10 ** 18)\
    .step_limit(1000000000) \
    .nonce(100) \
    .action("add") \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Other Contract Addresses

In [ ]:
pprint(contracts)

## Saving the pkl file


In [ ]:
now = datetime.utcnow()
filename ="reward_retest"+f'_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'

print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

## Updating contracts

In [ ]:
# Updating mainnet contracts
contract_ = 'rewardDistribution'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDS':
    update = contracts['oUSDS']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
elif contract_ == 'dICX':
    update = contracts['dICX']
    contract_ = 'dToken' 
elif contract_ == 'dUSDS':
    update = contracts['dUSDS']
    contract_ = 'dToken'
elif contract_ == 'dIUSDC':
    update = contracts['dIUSDC']
    contract_ = 'dToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

# Configuring SCOREs's parameters

In [ ]:
settings_lendingPool = [
    {'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
    {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
    {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
    {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
    {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
    {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
    {'contract': 'lendingPool', 'method': 'setRewardManager','params': {'_address':contracts['rewardDistribution']}},
    {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
    {'contract': 'lendingPool', 'method': 'setFeeSharingTxnLimit','params': {'_limit': 3}},
    {'contract': 'lendingPool', 'method': 'setBridgeOtoken', 'params': {'_address': contracts['oUSDS']}},
    {'contract': 'lendingPool', 'method': 'setOmmToken', 'params': {'_address': contracts['ommToken']}}
]
failed_methods = []
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(get_tx_result(tx_hash)['status'])
    if get_tx_result(tx_hash)['status'] != 1:
        failed_methods.append(sett["method"])
print(failed_methods)

In [ ]:
settings_feeProvider = [
    {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
    
]
for sett in settings_feeProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))

In [ ]:
if usd == "usds":
    symbol = "USDS"
elif usd == "usdb":
    symbol = "USDB"
settings_lendinPoolDataProvider=[
    {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts[usd],'_sym':symbol}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}}
]
failed_methods = []
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_oToken =[
    {'contract': 'oUSDS', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'oUSDS', 'method': 'setReserve', 'params':{'_address':contracts[usd]}},
    {'contract': 'oUSDS', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'oUSDS', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'oUSDS', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},
    {'contract': 'oUSDS', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]
failed_methods = []
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
setting_priceOracle =[
    {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
    {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
    {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'USD','_rate':10*10**17}}
]
failed_methods = []
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
setting_addressProvider =[  
    {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'addressProvider', 'method': 'setUSDs', 'params':{'_address':contracts[usd]}},
    {'contract': 'addressProvider', 'method': 'setoUSDs', 'params':{'_address':contracts['oUSDS']}},
    {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
    {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
    {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
    {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
    {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
    {'contract': 'addressProvider', 'method': 'setOmmToken', 'params':{'_address':contracts['ommToken']}},
    {'contract': 'addressProvider', 'method': 'setDelegation', 'params':{'_address':contracts['delegation']}},
    {'contract': 'addressProvider', 'method': 'setRewards', 'params':{'_address':contracts['rewardDistribution']}},
    {'contract': 'addressProvider', 'method': 'setdUSDs', 'params':{'_address':contracts['dUSDS']}},
    {'contract': 'addressProvider', 'method': 'setdIUSDC', 'params':{'_address':contracts['dIUSDC']}},
    {'contract': 'addressProvider', 'method': 'setdICX', 'params':{'_address':contracts['dICX']}}
]

failed_methods = []

for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_oicx = [
    {'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
    {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'oICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},
    {'contract': 'oICX', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]

failed_methods = []

for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_oiusdc = [ 
    {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'oIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},    
    {'contract': 'oIUSDC', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}}
]
failed_methods = []
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
pprint(contracts)

In [ ]:
# feeProvider is daoFund for now
settings_liquidationManager =  [
    {'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
    {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
    {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}},    
    {'contract': 'liquidationManager', 'method': 'setStaking','params': {'_address':contracts['staking']}}
]
failed_methods = []
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_delegation =[
    {'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
    {'contract': 'delegation', 'method': 'addAllContributors','params': {'_preps':PREP_LIST}},
    {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}},    
#     {'contract':  'delegation', 'method': 'setVoteThreshold','params': {'_vote':    }} # set on on_install()
]
failed_methods = []
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_lendingPoolCore =[ 
    {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
    {'contract': 'lendingPoolCore', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}},
    {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
    {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
    {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
    {'contract': 'lendingPoolCore', 'method': 'setGovernance','params': {'_address':contracts['governance']}},
    {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}}
]
failed_methods = []
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_priceOracle =[
    {'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':contracts['bandOracle']}},
    {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}
]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
pprint(contracts)

In [ ]:
# SNAPSHOT CONTRACT NOT USED
# settings_snapshot =[
#     {'contract': 'snapshot', 'method': 'setAdmin','params': {'_address':contracts['lendingPool']}},
#     {'contract': 'snapshot', 'method': 'setGovernance','params': {'_address':contracts['governance']}}
# ]
# for sett in settings_snapshot:
#     print(f'\n')
#     print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
#     transaction = CallTransactionBuilder()\
#         .from_(deployer_wallet.get_address())\
#         .to(contracts[sett['contract']])\
#         .value(0)\
#         .step_limit(10000000)\
#         .nid(NID)\
#         .nonce(100)\
#         .method(sett['method'])\
#         .params(sett['params'])\
#         .build()
#     signed_transaction = SignedTransaction(transaction, deployer_wallet)
#     tx_hash = icon_service.send_transaction(signed_transaction)
#     tx_hash
#     print(get_tx_result(tx_hash)['status'])

In [ ]:
# def setDistPercentage(self, _deposit: int, _borrow: int, _ommICX: int, _ommUSDb: int, _worker: int, _daoFund: int):
dist_params = { 
    "_ommICX": 1*10**17, 
    "_dex": 1*10**17, 
    "_worker": 2*10**17, 
    "_daoFund": 2*10**17}
setting_reward_percentage =[{'contract':'rewardDistribution','method':'setDistPercentage','params':dist_params}]
for sett in setting_reward_percentage:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))


In [ ]:
settings_rewards = [
            {'contract': 'rewardDistribution', 'method': 'setLendingPoolDataProvider',
             'params': {'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'rewardDistribution', 'method': 'setOmm',
             'params': {'_address': contracts['ommToken']}},
            {'contract': 'rewardDistribution', 'method': 'configureAssetEmission',
             'params': {
                "_assetConfig": 
                    [
                        {"asset": contracts["oUSDS"], "emissionPerSecond":f"{OUSDS_EMISSION}",'totalBalance': "0" },
                        {"asset": contracts["dUSDS"], "emissionPerSecond":f"{DUSDS_EMISSION}",'totalBalance': "0" },
                        {"asset": contracts["dICX"], "emissionPerSecond":f"{DICX_EMISSION}",'totalBalance': "0" },
                        {"asset": contracts["oICX"], "emissionPerSecond":f"{OICX_EMISSION}",'totalBalance': "0"}, 
                        {"asset": contracts["oIUSDC"], "emissionPerSecond":f"{OIUSDC_EMISSION}",'totalBalance': "0" },
                        {"asset": contracts["dIUSDC"], "emissionPerSecond":f"{DIUSDC_EMISSION}",'totalBalance': "0"} 
                    ]             
                }
            },
            {'contract': 'rewardDistribution', 'method': 'setWorkerToken',
             'params': {'_address': contracts['worker_token']}},
            {'contract': 'rewardDistribution', 'method': 'setAdmin',
             'params': {'_address': contracts['governance']}},
            {'contract': 'rewardDistribution', 'method': 'setDaoFund',
             'params': {'_address': contracts['daoFund']}},
            {'contract': 'rewardDistribution', 'method': 'setLendingPool',
             'params': {'_address': contracts['lendingPool']}},
            {'contract': 'rewardDistribution', 'method': 'setLpToken',
             'params': {'_address': contracts['dex']}},
            {'contract': 'rewardDistribution', 'method': 'setGovernance',
             'params': {'_address': contracts['governance']}}
]

failed_methods = []

for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

## Timestamp to be added 

In [ ]:
settings_governance =[
    {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewardDistribution']}},
    {'contract': 'governance', 'method': 'setStartTimestamp','params': {'_timestamp': TIMESTAMP}}, # REWARDS ma we need setStartTimestamp
    {'contract': 'governance', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}}
]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

## Initialize reserve in lending pool via governance

In [ ]:
# was in lending pool core, not in governance
settings_reserves = [{'contract': 'governance', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts[usd],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'governance', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
                    {'contract': 'governance', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['iusdc'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}}
                    ]

failed_methods = []

for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

In [ ]:
settings_ommToken =[
    {'contract':  'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewardDistribution']}},
    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewardDistribution']}},
    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':10 * 10**18}},
    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}
]
failed_methods = []
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

## Configure ICX Debt Token

In [ ]:
settings_dicx = [
    {'contract': 'dICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'dICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
    {'contract': 'dICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'dICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'dICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},
    {'contract': 'dICX', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]

failed_methods = []
for sett in settings_dicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

## Configure USDS Debt Token

In [ ]:
settings_dUSDS = [
    {'contract': 'dUSDS', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'dUSDS', 'method': 'setReserve', 'params':{'_address':contracts[usd]}},
    {'contract': 'dUSDS', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'dUSDS', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'dUSDS', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},    
    {'contract': 'dUSDS', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]
failed_methods = []
for sett in settings_dUSDS:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)

## Configure IUSDC Debt Token

In [ ]:
settings_diusdc =[
    {'contract': 'dIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'dIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
    {'contract': 'dIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'dIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'dIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},  
    {'contract': 'dIUSDC', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]

failed_methods = []

for sett in settings_diusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)
    

## Adding USDS reserve in LendingPoolCore via Governance



In [ ]:
# initially was in lending pool core, now governance controls it
params ={
    "_reserve": {
        "reserveAddress":contracts[usd],
        "oTokenAddress":contracts['oUSDS'],
        "dTokenAddress": contracts['dUSDS'],
        "lastUpdateTimestamp": "0",
        "liquidityRate":"0",
        "borrowRate":"0",
        "liquidityCumulativeIndex":f"1{'0'*18}",
        "borrowCumulativeIndex":f"1{'0'*18}",
        "baseLTVasCollateral":"500000000000000000",
        "liquidationThreshold":"650000000000000000",
        "liquidationBonus":"100000000000000000",
        "decimals":"18",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled":"1",
        "isFreezed":"0",
        "isActive":"1"
    } 
}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['governance']) \
    .nid(NID) \
    .nonce(100) \
    .method("initializeReserve")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Adding ICX reserve

In [ ]:
# initially was in lending pool core, now governance controls it
params ={
    "_reserve": {
        "reserveAddress":contracts['sicx'],
        "oTokenAddress":contracts['oICX'],
        "dTokenAddress": contracts['dICX'],
        "lastUpdateTimestamp": "0",
        "liquidityRate":"0",
        "borrowRate":"0",
        "liquidityCumulativeIndex":f"1{'0'*18}",
        "borrowCumulativeIndex":f"1{'0'*18}",
        "baseLTVasCollateral":"500000000000000000",
        "liquidationThreshold":"650000000000000000",
        "liquidationBonus":"100000000000000000",
        "decimals":"18",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled":"1",
        "isFreezed":"0",
        "isActive":"1"
    } 
}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['governance']) \
    .nid(NID) \
    .nonce(100) \
    .method("initializeReserve")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Adding IUSDC reserve

In [ ]:
# initially was in lending pool core, now governance controls it
params ={
    "_reserve": {
        "reserveAddress":contracts['iusdc'],
        "oTokenAddress":contracts['oIUSDC'],
        "dTokenAddress": contracts['dIUSDC'],
        "lastUpdateTimestamp": "0",
        "liquidityRate":"0",
        "borrowRate":"0",
        "liquidityCumulativeIndex":f"1{'0'*18}",
        "borrowCumulativeIndex":f"1{'0'*18}",
        "baseLTVasCollateral":"500000000000000000",
        "liquidationThreshold":"650000000000000000",
        "liquidationBonus":"100000000000000000",
        "decimals":"6",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled":"1",
        "isFreezed":"0",
        "isActive":"1"
    } 
}

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['governance']) \
    .nid(NID) \
    .nonce(100) \
    .method("initializeReserve")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Testing deposit

In [ ]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(5 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


## Testing borrow

In [ ]:
params ={"_reserve": contracts['sicx'], "_amount": 1 * 10 ** 18}

call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("borrow") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)